# Recommender System Implementation
To ensure a high quality baseline for our own understanding of recommender systems as well as our comparison, we began our evaluation of recommender systems under a variaty of matrix factorization steps for completing the user item matrix and finished by experimenting on how we can introduce hybrid filtering techniques in our recommendations. For our matrix factorization methods we implemented gradient descent, alternating least squares and the generalization of matrix factorization as released in 2010, factorization machines. Factorization machines were implemented mathematically and with a framework that could better compute the necessary elements, fastFM.

## Utilizing clustering
Under gradient descent and alternating least squares we use a user-item matrix $R$ which via clustering we experimented on. Two different types of the user-item ratings matrix $R$ were used for our baseline, one using user clustering and the other using item clustering. By setting up $R$ in this way we avoided challenges brought on by the binary nature of our data's ratings by increasing the maximum score a rating can achieve.

### Ratings calculations for Item and User Clusters
The ratings for user $u_i$ in an item cluster $IC$ is denoted as $IC_i$ and is calculated as the sum of all interactions that user $u_i$ had with various items in the cluster. Mathematically, it can be expressed as:
$$
IC_{i} = \sum_{j=1}^{n} I_{ij}
$$
The ratings for item $v_{j}$ in a user cluster $UC$ is denoted as $UC_{j}$ and is calculated as the sum of all interactions that users in the cluster had with item $v_{j}$. Mathematically, it can be expressed as:
$$
UC_{j} = \sum_{i=1}^{m} I_{ij}
$$

Under both user and item clustering:
- $n$ or $m$ represent the total number of items in a cluster,
- $I_{ij}$ is a binary indicator defined as follows $ I_{ij} = \begin{cases} 1, & \text{if user } u_{i} \text{ has interacted with item } v_{j} \\ 
0, & \text{otherwise} \end{cases}
$

# Ranking
Ranking is the step where we take a sparse set of ratings, $R$, and try to predict what unknown ratings are, $\hat{R}$.

For matrix factorization techniques, predicting ratings is formulated as the following non-convex optimization problem which seeks to minimize least squared error and use regularization to avoid overfitting:

$$\min_{U,V} \sum_{r_{ij} \text{observed}}{(r_{ij}-u_{i}^Tv_{j})^2} + \lambda(\sum_{i}\|u_i\|^2 + \sum_{j}\|v_j\|^2)$$ 

### Adding Features 
To utilize complex features we experimented with appending user and item features to their corresponding latent factor matrices, $U$ and $V$. Under ALS respective features were concatenated to the bottom of $U$ and $V$ whereas in SGD the features were concatenated to the right side.

## Gradient Descent
### Introduction
Under gradient descent, we create partial derivatives of the aformentioned function with respect to $u_i$ and $v_j$ to acquire update functions for each vector. Using these update functions each vector in U and V at indices corresponding to observed ratings are updated.

The update formulae for $u_i$ and $v_j$ with $i$ and $j$ corresponding to the row and column of an observed rating, are as follows:

$$
\begin{aligned}
u_i^{\text{new}} &= u_i + 2\alpha (r_{ij} -  v_j u_i^T)\cdot v_j - 2\alpha\lambda u_i\\
v_j^{\text{new}} &= v_j + 2\alpha (r_{ij} -  v_j u_i^T)\cdot u_i - 2\alpha\lambda v_j
\end{aligned}
$$

### Implementation
Our implementation of gradient descent for ratings matrix predictions is simple; iterate over the indices of all observed ratings updating corresponding rows in U and V at each rating. During this process we track the updates so that we can stop the training loop early if the algorithm starts to converge.

## Alternating Least Squares
### Introduction
Alternating Least Squares, or ALS, is another method of matrix factorization similar to gradient descent with one key difference; instead of updating by vector, entire matrices $U$ and $V$ alternate as a fixed variable while optimizing the other. Our implementation of ALS was based off of lecture 14 from CME 323: Distributed Algorithms and Optimization, Spring 2015 from Stanford. Under our implementation of ALS from Stanford the user and item matrices $U$ and $V$ both are of dimension $k$ x $n$ and $k$ x $m$ respectively. The complete ratings matrix $R$ is thus estimated via $\hat{R} = U^TV$. 

Compared to gradient descent, ALS is faster and requires less iterations. By fixing $U$ we obtain a convex function of $V$ and vice versa. Therefore in ALS we fix and optimize opposite matrices until convergence. Below is the general algorithm as described in the Stanford materials:

* Initialize $k$ x $n$ and $k$ x $m$ matrices $U$ and $V$
* Repeat the the following until convergence
    * For all column vectors $i = 1,... , n$    
    $$ u_i = (\sum_{r_{ij}\in r_{i *}}{v_jv_j^T + \lambda I_k})^{-1} \sum_{r_{ij}\in r_{i *}}{r_{ij}v_{j}}$$

    * For all column vectors $j = 1,... , m$
    $$ v_j = (\sum_{r_{ij}\in r_{* j}}{u_iu_i^T + \lambda I_k})^{-1} \sum_{r_{ij}\in r_{* j}}{r_{ij}u_{i}}$$

To break it down into pieces:
* $\sum_{{r_{ij}\in r_{i *}}} v_jv_j^T$ and $\sum_{r_{ij}\in r_{* j}} u_iu_i^T$ represent the sum of column vectors multiplied by their transpose where the vectors are determined by either the column vectors correspond to items that user $u_i$ has rated in $V$ or the column vectors correspond to the users in $U$ that have rated item $v_j$.
* $\lambda I_k$ represents the addition of a regularization term $\lambda$ to avoid overfitting.
* $\sum_{r_{ij}\in r_{i *}}{r_{ij}v_{j}}$ and $\sum_{r_{ij}\in r_{* j}}{r_{ij}u_{i}}$ represent the scaling of each column feature vector by a rating with indexing handled in the same way as $\sum v_jv_j^T$ and $\sum u_iu_i^T$.


### Implementation 
ALS implementation starts during the creation of the matrix $R$. Since ALS requires us to subset $V$ and $U$ for columns that correspond to items a user has rated or users that have rated an item we used several hash maps to store these indices. Hash maps were created during matrix initialization to take advantage of already iterating over items that a user has rated. With the matrix created and our maps initialized, we created $U$ and $V$ as random matrices with numbers drawn from a uniform distribution. For the optimization steps we found that $\sum_{r_{ij}\in r_{i *}}{v_jv_j^T}$ and $\sum_{r_{ij}\in r_{* j}}{u_iu_i^T}$ are the same as $V_jV_j^T$ and $U_iU_i^T$ with $U_i$, $U_j$, $V_j$ and $V_j^T$ being the subsets of $U$ and $V$ corresponding to observed ratings. However, this same process did not apply to $\sum_{r_{ij}\in r_{i *}}{r_{ij}v_{j}}$ and $\sum_{r_{ij}\in r_{* j}}{r_{ij}u_{i}}$, instead we found that we could multiply the observed ratings as a row vector by $V_j^T$ or $U_i^T$ and get the same result as taking the sum. Additionally to introduce regularization we added $\lambda$ multiplied by the $k$ x $k$ identity matrix to $U_iU_i^T$ and $V_iV_i^T$ while the matrices were being updated. 

Our final update functions for our matrices thus looked like:

$ u_i = ({V_jV_j^T + \lambda I_k})^{-1} {R_{i*}V_{j}^T}$


$ v_j = ({U_iU_i^T + \lambda I_k})^{-1} {R_{*j}U_{i}^T}$

## Factorization Machines
### Introduction
Introduced in 2010, factorization machines offered a generalization of matrix factorization methods. Factorization machines capture all single and pairwise interactions between variables with a closed model equation computable in linear time which is very effective. This is advantagous as it allows for the usage of stochastic gradient descent to learn model parameters. 

Factorization machines utilize high dimensional feature vectors along with a feature matrix denoted as $V$. For our implementation we implemented a factorization model of degree 2, which per the introductory paper on factorization machines __source__ has the following equation: 

$$ \hat{y}(x) := w_0 + \sum_{i=1}^{n}{w_i x_i} + \sum_{i=1}^{n}\sum_{j=i+1}^{n} \langle \bold{v}_i, \bold{v}_j\rangle x_i x_j$$

The model parameters that are estimated include $w_0$, $w$ and $V$, where $w_0$ represents global bias, $w$ represents the weights of all possible features in a feature vector $x$ and $V$ is an $n$ x $k$ feature matrix. Pairwise interactions are modeled by $\langle \bold{v}_i, \bold{v}_j\rangle.$

A row within the feature matrix $V$ is defined as $v_i$ which describes the i-th feature with $k$ factors where $k$ represents the dimensionality of the factorization. 

### Gradient Descent for Factorization Machines
As mentioned earlier, model parameters $w_0$, $w$ and $V$ can all be learned via gradient descent methods on a variety of losses. As a result, we utilized stochastic gradient descent to optimize and tune our model parameters with our data. Below is the gradient vector of the function $\hat{y}$ for the estimated model parameters.
$$\frac{\partial}{\partial\theta}\hat{y}(x) = \begin{cases} 1, & \text{if } \theta \text{ is } w_0 \\ x_i, & \text{if } \theta \text{ is } w_i \\ x_i\sum_{j=1}^{n}{v_{j,f}}x_j - v_{i,f}x_i^2, & \text{if } \theta \text{ is } v_{i,f} \end{cases} $$

Additionally, to stay consistent in our judgement of our baseline models, we focused on minimizing residuals under the squared loss function, $(y - \hat{y})^2$, which is shown below: 

 $$(y -  (w_0 + \sum_{i=1}^{n}{w_i x_i} + \sum_{i=1}^{n}\sum_{j=i+1}^{n} \langle \bold{v}_i, \bold{v}_j\rangle x_i x_j))^2$$

Given the gradient vector and standard loss we create the following update formulae: 

$w_0^{new} = w_0 + \alpha2(y - \hat{y})$

$w_i^{new} = w_i + \alpha2(y - \hat{y}) * x_i $

$v_{i,f}^{new} = v_{i,f} + \alpha2(y - \hat{y}) * x_i\sum_{j=1}^{n}{v_{j,f}}x_j - v_{i,f}x_i^2$

### Implementations

We implemented two versions of factorization machines, one by hand using the gradients as described above and another using a factorization machine framework fastFM. We utilized our own implementation on a small subset of the data to ensure we have an understanding of how gradient descent can be applied to different optimization problems. The fastFM implementation was used to test out the effectiveness of a factorization machine on our entire dataset. 

#### Our Own Implementation
Factorization machines facilitate the usage of high dimensional feature vectors meaning proper management of large data becomes a priority. Our solution to the issue of memory was a sparse matrix that was created from the Tensorflow compatible dataset to hold feature vectors efficiently in memory. Feature vectors included information about the user, their interaction, rating, previous interactions, median time of day of interactions, item features and personal taste. Model parameters $V$, $w_0$ and $w$ were initialized randomly with samples from a uniform distribution. Parameters were updated at different cadences as every feature vector contained multiple $w_i$ and $v_{i,f}$, meaning $w_0$ was updated once every feature vector and $w_i$ and $v_{i,f}$ were updated for every instance of a feature within a single feature vector. 

In [ ]:
import pandas as pd
import numpy as np
import matrix_modules

In [ ]:
def y_hat(w_0, x, w, features_matrix):
    """ 
    Calculates the predicted error for the given parameters: w_0, x, w and the subset of V corresponding to the features present in the feature vector.

    Args:
        w_0 (int) : w_0 represents the global bias term added at the beggining of the y_hat calculation.
        x (dict) : The feature vector corresponding to a particular row index. x has two keys, indices and scores, which signify the indices in V and w that have scores and the corresponding scores.
        w (np.ndarray) : w represents the vector of feature weights and is a 1 x num_features dimensional row vector.
        features_matrix (np.ndarray) : features_matrix represents the subsetted matrix of V corresponding to the indices containing values in x.
    
    Returns:
        y_hat (int) : A predicted score for the feature vector.
    """

    # Get the feature indices and their corresponding scores from the feature vector x.
    feature_indices = x["indices"]
    scores = x["scores"]

    # Get the \sum{i=1}^{n}{w_ix_i} term.
    scaled_feature_weights = w[feature_indices] * scores
    
    # Initialize a total for the summation of inner product of pairwise rows in V with scores.
    total = 0

    # Get the number of rows in the subset of the original features matrix for looping.
    rows, _ = features_matrix.shape

    # Loop through all pairwise groups of rows finding their innner product and multiplying by their scores, summing the whole thing.
    for row_1 in range(rows):
        for row_2 in range(rows):
            # Check to see if one row is the same as another as we wish to avoid that case. 
            if row_1 == row_2:
                pass
            else:
                # Add the full calculation to total.
                # Subsetting scores in this way works because the number of scored items directly corresponds to the number of rows in the subset of our feature matrix V.
                total += scores[row_1] * x[row_2] * np.inner(features_matrix[row_1, :], features_matrix[row_2, :])
                
    # Return the linear combination of what we calculate to get the score prediction.
    return w_0 + total + scaled_feature_weights

In [ ]:
def update_w_0(w_0, err, alpha):
    """
    Update the global bias term using the error and the learning rate alpha.
    """
    return w_0 + 2 * alpha * err

In [ ]:
def update_w_i(x_i_score, w_i, err, alpha):
    """
    Update an index in the feature vector w using the the score in the feature vector, its weight in w, the error and the learning rate alpha.

    Args:
        x_i_score (int) : The score in the feature vector x corresponding to the feature with weight w_i.
        w_i (int) : The weight of the feature in w.
        err (int) : The error calculated by finding y - y_hat.
        alpha (int) : The learning rate that was chosen at model creation. 

    Returns:
        w_i (int) : The updated model weight at the ith index in w.
    """
    return w_i + 2*alpha*err*x_i_score


In [ ]:
def update_v_ij(x, v_ij, subset, row_i, err, alpha):
    """
    Updates the ith vector in the feature matrix V.

    Args:
        x (np.ndarray) : The row vector containing scores from a feature vector. 
        v_ij (np.ndarray) : A row vector from the feature matrix V. 
        subset (np.ndarray) : The subset of V corresponding to the indices of interacted with features.
        row_i (int) : The row index in the subset that is being updated.
        err (int) : The error calculated by finding y - y_hat.
        alpha (int) : The learning rate that was chosen at model creation. 
    
    Returns:
        Updated v_ij, the updated row vector in V.
    """
    # Get the number of rows in the subset to determine number of looping iterations.
    rows, k = subset.shape

    # Initialize a total to keep track of the sum.
    total = np.zeros((1, k))

    # Start looping over the rows of the subset not corresponding to the row determined by row_i.
    for j in range(rows):
        if j == row_i:
            pass
        else:
            total += subset[j, :] * x[j] - subset[row_i, :] * x[row_i]**2

    return v_ij + 2 * alpha * err * x[row_i] * total


In [ ]:
def factorization_machine(feature_vectors, k, num_features, alpha):
    """
    Takes in a set of feature vectors, the desired number of latent factors and the learning rate and then
    performs gradient descent in the vain of a factorization machine to train model parameters w_0, w and V.

    Args:
        feature_vectors (dict) : The sparse representation of feature vectors as a dictionary with keys 'indices' and 'scores'
        k (int) : The desired number of latent factors. 
        num_features (int) : The maximum index in the row vectors which gets used in the creation of V.
        alpha (int) : The learning rate for the gradient descent.
    
    Returns:
        w_0, w and V which after running will be trained model weights that can be used on new observations.
    """

    # Initialize w_0.
    w_0 = 1
    
    # Initialize w.
    w = np.random.uniform(0, 1, size=num_features).reshape((1, num_features))
    w_old = np.zeros_like(w)

    # Initialize V
    V = np.random.uniform(0, 1, size=num_features*k).reshape((num_features, k))
    V_old = np.zeros_like(V)

    # Iterate through all rows provided by the feature feature vectors argument.
    for row in range(len(feature_vectors)):

        # Get the indices of the features that are used and scores.
        indices = feature_vectors[row]["indices"]
        scores = feature_vectors[row]["scores"]
        rating = feature_vectors[row]["rating"]
        # Subset V for the rows corresponding to rated feature indices.
        V_subset = V[indices, :]

        # Calculate y_hat.
        rating_estimate = y_hat(w_0, row, w, V_subset)
        error =  rating - rating_estimate

        for index in range(len(scores)):
            # We first update the ith weight in w.
            w[:, indices[index]] = update_w_i(scores[index], w[:, indices[index]], error, alpha)

            # We then update the ith row of the feature matrix V.
            V[indices[index], :] = update_v_ij(scores, V[indices[index], :], V_subset, index, error, alpha)

        w_0 = update_w_0(w_0, error, alpha)
    
    return w_0, w, V


## Testing
Starting off with testing we evaluate the performance of stochastic gradient descent, alternating least squares, and factorization machines. Factorization machines were implemented with an existing framework ___ in order to save time on fine tuning linear algebra for compute efficiency.

# Making Recommendations

Arguably the other most important part of a recommender is the part that makes recommendations with the predicted ratings. The other largest sub-system of any recommender system is one that takes action on predicted ratings to make recommendations. What's considered a 'good' recommendation differs greatly depending on the goals in place for the recommender system as a whole. For example, if there are business incentives to promote articles of a certain category then 'good' recommendations are ones that a user will interact with and that further business goals. To ensure that our evaluation was comprehensive, we created a basic system to make recommendations on which we improved upon. 

## Metrics of Success
Recommendations can be evaluated by several success metrics: novelty, coverage and serendipity to name a few. Novelty is a measure of the newness of a recommendation. Coverage is a measure of how much of the catalogue is represented in recommendations. Serendipity measures both the newness of recommendations and how exciting they are. An example of a serendipitous recommendation would be if a user generally reads stories about sports in the united states and is then recommended a story about a lesser known sport from a different country they were unaware of. Improving and evaluating the serendipity of recommendations is both difficult and not agreed upon, therefore we chose the following ____ (research serendipity calculations) 

### Implementation of metrics

* To understand the coverage of our recommendations, we took the summation of all recommendations made and then determined the portion of the overall catalogue that was comprised of the recommendations. 
* To calculate the novelty of our recommendations, we utilized a vector similarity of item features to determine how similar recommended items were to those that were previously rated in addition to the popularity of articles.
* To calculate serendipity, we ___  

### Considering Edge Cases
In addition to improving the quality of recommendations in the directions that we deemed 'good' we also had to consider a couple different edge cases to improve the robustness of our systems. One such edge case that we explored was how we might deal with users that have ranked every available item, users that haven't ranked any items, or ones without many features to go off of.

### Basic Implementation of Recommender Step
For matrix factorization methods like gradient descent and alternating least squares, the most simple form of retrieval is to look at the highest n ratings present in a row of $\hat{R}$. This way of providing recommendations while simple is prone to providing recommendations of previously viewed items to users which immediately decreases its quality. 

### Improvements
Improving the quality of recommendations from ratings created by matrix factorization methods like ALS and SGD is a varied process. Since there's no 'best' way to recommend items we focused on improving recommendations in a few contexts: improving recommendations for a specific business goal, making recommendations more personal and adding another step to make intra-cluster recommendations. 

As mentioned previously, one of our first goals for improving recommendations was to avoid recommending items that a user has already rated. The way we implemented this was by utilizing the hash maps that are made during matrix creation to easily subset out viewed items from the set of recommendations prior to sorting it for the highest ratings.




### Making Recommendations : Utilizing Item Features

Current thoughts is leaning towards some weighted sum score for each item, like a feature similarity + dissimilarity + predicted ratings thingaling 

One way that we improved the quality of recommendations was to utilize both latent factors and given features from users and items to  
We further experimented on improving recommendations by utilizing the existing features alongside latent factors generated during matrix factorization for both users and items. Features were placed inside of several KNN models where a vector similarity could be used on highly rated and seen items to introduce the usage of features in similarity. User features were used to get the similar users and find recommendations that way.

In [2]:
# Import necessary libraries and load functions
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import matrix_modules

def calculate_coverage(items_recommended, items_present):
    """ 
    Calculates the coverage of recommendations made.

    Args:
        items_recommended (int) : The total number of items that are recommended to users.
        items_present (int) : The total number of items present in the catalog.

    Returns:
        coverage (float) : Returns the percent coverage of the catalog.
    """

    if items_recommended > items_present:
        return 1   
    else:
        return items_recommended / items_present

def calculate_novelty(recommendations, item_features, max_popularity):
    """
    Calculates the novelty of recommendations. Since novelty can be a measure of how popular items are 
    we calculate novelty by 1 - popularity(i) / max popularity
    """
    # Here we would just want to subset the recommendations made in the news features for their popularity and do calculations off of that
    # Iterate over all recommendations that are being made
    novelty_scores = []
    for recommendation in recommendations:

        # Recommendation is an index so we subset the item features for that index and get the popularity there 
        novelty_scores.append(1 - (item_features.loc[recommendation, 'popularity'] / max_popularity))

    return novelty_scores

def filer_recommendations(sorted_indices, viewed_items):
    """
    Filters the recommendations to exclude those that have already been rated by the user
    """
    # Create an empty list to populate with recommendable indices.
    recommendable_items = []

    # Sort through all indices that are recommendable.
    for index in sorted_indices:

        # If the index corresponds with one that has not already been viewed, add it to the list of recommendable items.
        if index not in viewed_items:
            recommendable_items.append(index)
    
    # If the recommendable items list is empty, return the sorted indices. 
    if not recommendable_items:
        recommendable_items = sorted_indices
        print("User has interacted with all possible items")        
    
    # If the recommendable items list is not empty, return it.
    return recommendable_items

In [32]:

user_features = pd.read_csv("../MIND_large/csv/user_features.csv", index_col=0)
item_features = pd.read_csv("../MIND_large/csv/item_features.csv", index_col=0).drop(columns=["Unnamed: 0", "travel.1"])

    
    
# argueably it is best to just have the features sorted and in their proper place before any of these steps. 
    # how do we go about that?
    # step 1 is to manage item features
    # we want to pass the category columns into an alphabetical sort
    # take a subset
    # remove the columns
    # attach the subset

cols = item_features.iloc[:, 4:-1].columns
cols.sort_values()
item_features = item_features.drop(columns=['title', 'abstract'])

item_clusters = item_features['cluster'] 
item_popularity = item_features['popularity']
item_ids = item_features['news_id']


user_clusters = user_features['cluster']
user_median = user_features['median_time']
user_ids = user_features['user_id']

item_features = item_features[cols]
user_features = user_features[cols]
# [item_ids, item_clusters, item_popularity, item_features]
# [user_ids, user_clusters, user_median, user_features]
item_features = pd.concat([item_ids, item_clusters, item_popularity, item_features], axis=1)
user_features = pd.concat([user_ids, user_clusters, user_median, user_features], axis=1)

# set(item_features.columns) == set(user_features.columns)  
item_features.head()
    

,news_id,cluster,popularity,autos,entertainment,finance,foodanddrink,games,health,kids,...,voices,watch,weatherfullscreenmaps,weathertopstories,weight-loss,weightloss,wellness,wines,wonder,yearinoffbeatgoodnews
0,N88753,11,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,N23144,16,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,N86255,22,8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,N93187,3,221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,N75236,5,1525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
user_features.head()

,user_id,cluster,median_time,autos,entertainment,finance,foodanddrink,games,health,kids,...,voices,watch,weatherfullscreenmaps,weathertopstories,weight-loss,weightloss,wellness,wines,wonder,yearinoffbeatgoodnews
0,U1,37,0.375000,0.000000,0.236111,0.111111,0.000000,0.0,0.000000,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,U100,43,0.250000,0.069767,0.046512,0.046512,0.116279,0.0,0.116279,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,U1000,37,0.375000,0.000000,0.000000,0.000000,0.111111,0.0,0.000000,0.0,...,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,U10000,16,0.291667,0.011494,0.022989,0.137931,0.264368,0.0,0.114943,0.0,...,0.02381,0.0,0.0,0.000000,0.0,0.0,0.011905,0.0,0.0,0.0
4,U100005,35,0.500000,0.000000,0.000000,0.043011,0.225806,0.0,0.021505,0.0,...,0.00000,0.0,0.0,0.032258,0.0,0.0,0.010753,0.0,0.0,0.0


In [19]:
n_neighbors = 5

item_features_knn = NearestNeighbors(n_neighbors=n_neighbors, metric = 'euclidean')
user_features_knn = NearestNeighbors(n_neighbors=n_neighbors, metric = 'euclidean')

user_features_knn.fit(ordered_clustered_user_features)
item_features_knn.fit(clustered_item_features)

,autos,entertainment,finance,foodanddrink,games,health,kids,lifestyle,middleeast,movies,...,voices,watch,weatherfullscreenmaps,weathertopstories,weight-loss,weightloss,wellness,wines,wonder,yearinoffbeatgoodnews
0,280.529947,478.449186,854.037346,563.767188,0.000000,768.570183,0.041435,1971.312133,0.157738,584.450805,...,89.495757,0.000000,10.240248,215.367745,1.267676,44.872551,118.823446,1.067144,10.189553,0.740745
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,1.733333,0.000000,0.000000,0.000000,0.250000,0.000000,203.016667,0.000000,0.000000,...,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,164.895107,111.214722,472.535785,172.812047,0.000000,160.230607,0.031577,322.322218,0.141050,208.439166,...,16.353693,0.000000,2.712865,76.589127,0.449970,7.303367,22.956029,0.459909,0.152907,0.329157
4,393.411341,341.618276,525.890091,471.424912,0.000000,386.617419,0.000000,834.994885,0.000000,478.892240,...,30.693513,0.000000,3.127834,78.284130,0.693050,32.901661,68.215383,0.340342,0.612286,0.139925
5,269.723528,153.644798,589.809823,313.628056,0.000000,345.841696,0.283851,740.894392,0.071693,288.562666,...,41.314190,0.000000,5.983306,129.891428,0.185839,16.991315,46.364014,0.444102,2.131246,0.975839
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,220.595107,1165.884049,632.930899,537.908384,0.000000,777.145966,0.023810,2555.700197,0.192861,1080.955634,...,99.338633,0.000000,5.809189,104.431372,1.834982,73.278086,164.884716,0.419870,1.394562,0.253642
8,0.055556,0.550000,0.135747,0.000000,0.000000,0.000000,0.000000,0.225490,0.000000,0.000000,...,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,89.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
clustered_item_features = item_features.drop(columns=['news_id', 'title', 'abstract']).groupby("cluster").agg(sum).reset_index()
cluster_popularity = clustered_item_features['popularity']
clustered_item_features = clustered_item_features.drop(columns=['popularity', 'cluster'])
ordered_user_features = user_features[item_features.iloc[:, 4:-1].columns]
median_times = user_features['median_time']
user_similarity = pd.concat([median_times, ordered_user_features], axis=1)

user_features_knn = NearestNeighbors(n_neighbors=n_neighbors, metric = 'euclidean')
item_features_knn = NearestNeighbors(n_neighbors=n_neighbors, metric = 'euclidean')

user_features_knn.fit(user_similarity)
item_features_knn.fit(clustered_item_features)

NearestNeighbors(metric='euclidean')

In [ ]:
# Set up the ratings matrix
print("Loading Dataset")
full_ratings, news, users = matrix_modules.load_dataset_for_matrix()

print("Getting Ratings Matrix")
R, item_idx, user_idx = matrix_modules.create_item_cluster_mat(full_ratings, news, num_users=len(users), isALS=True, num_clusters=len(news['cluster'].unique()))
item_idx = {num : sorted(list(users)) for num, users in item_idx.items()}
user_idx = {user_id : sorted(list(ratings)) for user_id, ratings in user_idx.items()}
seen = {user_id : set(ratings) for user_id, ratings in user_idx.items()}
K = 5
I = len(user_idx) 
M = len(item_idx)
U = np.random.uniform(0, 1, size=K*I).reshape((K, I))
V = np.random.uniform(0, 1, size=K*M).reshape((K, M))
print("Starting ALS")
U, V, track_error, track_update = matrix_modules.alternating_least_squares(U, V, R, user_idx, item_idx, max_iterations=10)
R_hat = U.T @ V

In [ ]:
# Initialize knn models for item features / factors
item_features_knn = NearestNeighbors(n_neighbors=n_neighbors, metric = 'euclidean')
item_factors_knn =  NearestNeighbors(n_neighbors=n_neighbors, metric = 'euclidean')

# Fit the models with the item feature data, drop the cluster column for the inputs into knn
item_features_knn.fit(grouped_item_features.drop(columns="cluster"))
item_factors_knn.fit(V.T);


In [ ]:
# Setting up the columns of user features to match the grouped item features
reordered_user_feature_colulmns = user_features[grouped_item_features.iloc[:, 1:].columns]
user_features = pd.concat([user_features.iloc[: , :1], reordered_user_feature_colulmns], axis=1)

In [ ]:
user_features

In [ ]:
def collaborative_filter(R, user_index, ratings_df, user_features, user_similarity_knn, seen):
    """
    Uses vector similarity to find similar users and their corresponding indices in the ratings matrix
    
    Args:
        user_index (int) : A row index in the ratings matrix corresponding to a user.
        ratings_df (pd.DataFrame) : The dataframe of ratings grouped by user id loaded in with load_dataset_for_matrix()
        user_similarity_knn (sklearn.neighbors.Knearestneighbors (something)) : The knn model fit on user vectors to pull similar indices from.

    Returns:
        similar_users (list(int)) : A list containing similar user indices in the ratings matrix.
    
    """
    # Get the users row from the item features data frame.
    user_row = user_features[user_features['user_id'] == ratings_df.iloc[user_index, 0]]

    # Get the indices of similar users in the ratings matrix. 
    _, indices = user_similarity_knn.kneighbors(user_row.drop(columns=['user_id']))

    # Initialize an empty set to hold onto item indices.
    items_to_rec = set()

    # Iterate over all other user indices.
    for idx in indices[0][1:]:
        
        # Get that users sorted items by using the seen dictionary.
        sorted_rating_indices = np.argsort(R[idx, list(seen[idx])], axis=0)[::-1]

        # For each item, add it to the set of items to recommend.
        for item in sorted_rating_indices[:5]:
            items_to_rec.add(item)
    
    # Return items to recommend.
    return items_to_rec

# need to update the filtering step to ensure that theres at least 5 items, even if already seen

def content_filter(user_idx, seen, item_features):
    """
    Introduces content filtering into the recommendation step by finding similar items to the ones already rated by the user.
    """
    items_to_rec = set()
    rated = list(seen[user_idx])
    for item_idx in rated:
        _, indices = item_features_knn.kneighbors(item_features.iloc[item_idx, 1:].to_frame().T)
        for index in indices[0][1:]:
            items_to_rec.add(index)
    
    return items_to_rec

def calculate_weights(user_feature_vector, item_indices, item_features):
    """
    uses user and item features to calculate the weights to use in filtering techniques
    """
    # Here we take our feature vector which is a dataframe input and then locate its row excluding the user_ID column,
    # then we make it into a numpy array and call the reshape method to make it two dimensional, for now it is a 256, 1 shape, then we call its transpose so we get a 1, 256 shape
    to_npy = user_feature_vector.iloc[0, 1:].to_numpy().reshape(-1, 1).T
    weights = []
    for item_index in item_indices:
        weight = to_npy @ item_features.iloc[1, 1:].to_numpy().reshape(-1, 1)
        print(item_features.iloc[item_index, 1:].to_frame().T)
        print(user_feature_vector)
        # item_features.iloc[item_idx, 1:].to_frame().T

def get_top_N_ratings(row_index, seen=None):
    """
    Gets the top N ratings in a row of the ratings matrix.
    """

def recommend_items(R, user_index, ratings_df, user_features, item_features, user_similarity_knn, seen):
    """ 
    The main function that combines all other recommendation step stuff.
    """
    collaborative_items = collaborative_filter(R, user_index, ratings_df, user_features, user_similarity_knn, seen)
    content_items = content_filter(user_index, seen, item_features)
    print(collaborative_items)
    print(content_items)


In [ ]:
recommend_items(R, 1, full_ratings, user_features, item_features, user_features_knn, seen)

In [ ]:
feature_row = user_features[user_features['user_id'] == 'U1']
feature_row

In [ ]:
gif_wo_pop.iloc[0, :].to_frame().T

In [ ]:
feature_row

In [ ]:
feature_row.iloc[0, 1:].to_numpy().reshape(-1, 1).T @ gif_wo_pop.iloc[1, 1:].to_numpy().reshape(-1, 1)
# here we use both 1: for feature row and gif_wo_pop because that allows for us to avoid getting the user id or cluster in our results

In [ ]:
calculate_weights(feature_row)

In [ ]:
def calculate_ratings_weights(items, user_features_row, item_features):
    """ 
    Calculates ratings weights by taking the dot product of the user feature vector with each items feature vector.
    Or we could just consider taking the product of the users preference score with the category and then do 
    something with embeddings maybe?

    item_features (rows of itme feature dataframe forming a subset for interacted with items)
    user_features_row (the row of user preferences)
    """
    # need to look at the feature table for the item indices and make sure that everything is lining up
    # need to also make sure that item indices are all fine for user clustering methods 
    # then also since we have these latent factors of k dimension we can even consider doing a UMAP reduction of the K factors to
    # fit the number of features and then utilize those for recommendations as well maybe? 
    weights = []
    for row_index in range(len(item_features)):
        # user_features_row['']
        # Basically we just get the items category
        item_row = item_features.iloc[row_index, :]
        item_row = item_row[item_row != 0] # need to test this
        category = item_row['category'].columns#? # need to subset where the value is 1

        # Then the weight becomes the users preference for that category
        user_features_row[category]
        # then in regards to opportunities for embeddings, we could get all embeddings that a user prefers and then do something with that in regards
        # to the embeddings of the article? 
        # And then a popularity weighting?
        # maybe we want to include some sort of inverse weighting penalty system where we subtract the vector similarity from the weight?
        # weights.append()

    

In [ ]:
def get_similar_user_ratings(indices, R_hat):
    """
    Takes in a list of indices corresponding to users for similar features.
    """
    
    for index in indices:
        row = R_hat[index, :]
    


In [ ]:
# Now for similar users we iterate through their highest rated items and add them to the set of filtered recommendations
# Or maybe we need to implement a function that utilizes item features and the users preferences to get the highest quality
# recommendations when fed a set of indices




In [ ]:
# now we consider how we would combine user features and item features / factors
# we could always just take their feature preferences w/o the timestamp and make sure that the order of elements is the same ?
# like we could take the vector of preferences u and the vector of item features and then take their dot product and that would 
# be considered a weight on the users predicted score for the item. 
# We also might want to dimension reduce or make it so the preferences are the same